# Importing Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import seaborn as sns
sns.set()
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf 
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from collections import  Counter
import nltk
nltk.download('stopwords')
import re

from collections import defaultdict
from sklearn.preprocessing import LabelEncoder, StandardScaler,OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from xgboost import XGBRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from scipy import stats
from scipy.stats import norm, skew 
from scipy.special import boxcox1p
from sklearn.preprocessing import RobustScaler

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing the Data

In [ ]:
data = pd.read_csv('../input/water-potability/water_potability.csv')

In [ ]:
data

# Checking for Missing values

In [ ]:
missing_value = data.isnull().sum().sort_values(ascending = False)
missing_perc = (data.isnull().sum()*100/data.shape[0]).sort_values(ascending = False)
value = pd.concat([missing_value,missing_perc],axis=1,keys=['Count','%'])
display(value.head(20).style.background_gradient(cmap = 'Reds', axis = 0))

The Sulfate, PH and Trihalomethanes have missing values.

In [ ]:
data.describe(include='all')

In [ ]:
# Filling up the null values with the median
data = data.apply(lambda x: x.fillna(x.median()),axis=0)

In [ ]:
data.describe(include='all')

# Properties V/S Potability

In [ ]:
fig = plt.figure(figsize=(18,15))
gs = fig.add_gridspec(3,3)
ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1])
ax2 = fig.add_subplot(gs[0,2])
ax3 = fig.add_subplot(gs[1,0])
ax4 = fig.add_subplot(gs[1,1])
ax5 = fig.add_subplot(gs[1,2])
ax6 = fig.add_subplot(gs[2,0])
ax7 = fig.add_subplot(gs[2,1])
ax8 = fig.add_subplot(gs[2,2])
sns.kdeplot(ax=ax0,data = data, x= 'ph',hue='Potability',fill=True)
sns.kdeplot(ax=ax1,data = data, x= 'Hardness',hue='Potability',fill=True)
sns.kdeplot(ax=ax2,data = data, x= 'Solids',hue='Potability',fill=True)
sns.kdeplot(ax=ax3,data = data, x= 'Chloramines',hue='Potability',fill=True)
sns.kdeplot(ax=ax4,data = data, x= 'Sulfate',hue='Potability',fill=True)
sns.kdeplot(ax=ax5,data = data, x= 'Conductivity',hue='Potability',fill=True)
sns.kdeplot(ax=ax6,data = data, x= 'Organic_carbon',hue='Potability',fill=True)
sns.kdeplot(ax=ax7,data = data, x= 'Trihalomethanes',hue='Potability',fill=True)
sns.kdeplot(ax=ax8,data = data, x= 'Turbidity',hue='Potability',fill=True)

# Correlation Map

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(10, 10))
df_corr = data.corr()
sns.heatmap(df_corr,fmt=".2f",annot=True,cmap='YlGnBu',ax=ax)


# Preprocessing the data

In [ ]:
from sklearn.preprocessing import MinMaxScaler
X = data.drop('Potability',axis=1)
y = data['Potability']

scaler = MinMaxScaler()
X[X.columns] = scaler.fit_transform(X)


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size = 0.3)


# Determining the Potability using various Classifiers

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

gaussian = GaussianNB()
gaussian.fit(X_train, y_train)
y_pred = gaussian.predict(X_valid)
acc_gaussian = round(accuracy_score(y_pred, y_valid) * 100, 2)
print(acc_gaussian)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

randomforest = RandomForestClassifier()
randomforest.fit(X_train, y_train)
y_pred = randomforest.predict(X_valid)
acc_randomforest = round(accuracy_score(y_pred, y_valid) * 100, 2)
print(acc_randomforest)


In [ ]:
from sklearn.svm import SVC
svm = SVC()
svm.fit(X_train, y_train)
y_pred = svm.predict(X_valid)
acc_svm = round(accuracy_score(y_pred, y_valid)*100,2)
print(acc_svm)

In [ ]:
#Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier

gbk = GradientBoostingClassifier()
gbk.fit(X_train, y_train)
y_pred = gbk.predict(X_valid)
acc_gbk = round(accuracy_score(y_pred, y_valid) * 100, 2)
print(acc_gbk)

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
lo = logreg.fit(X_train,y_train)
y_pred = lo.predict(X_valid)
acc_log = round(accuracy_score(y_pred, y_valid) * 100, 2)
print(acc_log)

# Result

In [ ]:
result = pd.DataFrame({'Model' : ['Logistic Regression', 'RandomForest', 'Gradient Boosting', 
                         'SVM', 'GaussianNB'],
                         'Score' : [acc_log, acc_randomforest, acc_gbk, acc_svm, acc_gaussian]})

result.sort_values(by='Score', ascending=False, inplace=True)

In [ ]:
sns.barplot(y = 'Model',x = 'Score', data = result)

**Have Got around 70% accuracy using RandomForest Classifier.**